In [1]:
"""
finalize topic modeling of r/technews posts

try additional stop words to remove company names
so topics are about themes in technology rather
than topics by company
"""

'\nfinalize topic modeling of r/technews posts\n\ntry additional stop words to remove company names\nso topics are about themes in technology rather\nthan topics by company\n'

In [1]:
import pandas as pd
import numpy as np
import pickle
import datetime

import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [2]:
with open('preprocessed_posts.pickle', 'rb') as read_file:
    posts = pickle.load(read_file)
    
posts.shape

(48082, 18)

In [3]:
posts.head()

,id,created_utc,date_time,author,author_fullname,title,url,domain,num_comments,score,permalink,full_link,refine_title,preproc_title,unstemmed,stem_title,processed_title,title_nouns
0,29j6ov,1404184792,2014-07-01 03:19:52,Password_is_123456,t2_6crtu,New leaks show off the iPhone 6's curved edged...,http://www.cnet.com/news/new-leaks-show-off-up...,cnet.com,0,0,/r/technews/comments/29j6ov/new_leaks_show_off...,https://www.reddit.com/r/technews/comments/29j...,New leaks show off the iPhone 6's curved edged...,New leaks show off the iPhone s curved edged...,"[New, leaks, show, off, the, iPhone, s, curved...","[new, leak, show, off, the, iphon, s, curv, ed...",new leak show off the iphon s curv edg display,New leaks iPhone s display
1,29j6op,1404184791,2014-07-01 03:19:51,Password_is_123456,t2_6crtu,Millions of dynamic DNS users suffer after Mic...,http://arstechnica.com/security/2014/06/millio...,arstechnica.com,6,63,/r/technews/comments/29j6op/millions_of_dynami...,https://www.reddit.com/r/technews/comments/29j...,Millions of dynamic DNS users suffer after Mic...,Millions of dynamic DNS users suffer after Mic...,"[Millions, of, dynamic, DNS, users, suffer, af...","[million, of, dynam, dns, user, suffer, after,...",million of dynam dns user suffer after microso...,Millions DNS users Microsoft IP domains
2,29j2c1,1404181722,2014-07-01 02:28:42,[deleted],NaN,Paypal Freezes ProtonMail Campaign Funds,https://protonmail.ch/blog/paypal-freezes-prot...,protonmail.ch,0,1,/r/technews/comments/29j2c1/paypal_freezes_pro...,https://www.reddit.com/r/technews/comments/29j...,Paypal Freezes ProtonMail Campaign Funds,Paypal Freezes ProtonMail Campaign Funds,"[Paypal, Freezes, ProtonMail, Campaign, Funds]","[paypal, freez, protonmail, campaign, fund]",paypal freez protonmail campaign fund,Paypal Freezes ProtonMail Campaign Funds
3,29hy6f,1404156644,2014-06-30 19:30:44,ANIMAL_NewYork,t2_h1mjr,UNETHICAL FACEBOOK “EMOTIONAL CONTAGION” STUDY...,http://animalnewyork.com/2014/facebook-emotion...,animalnewyork.com,0,1,/r/technews/comments/29hy6f/unethical_facebook...,https://www.reddit.com/r/technews/comments/29h...,UNETHICAL FACEBOOK “EMOTIONAL CONTAGION” STUDY...,UNETHICAL FACEBOOK “EMOTIONAL CONTAGION” STUDY...,"[UNETHICAL, FACEBOOK, “EMOTIONAL, CONTAGION”, ...","[uneth, facebook, “emot, contagion”, studi, no...",uneth facebook “emot contagion” studi not fund...,FACEBOOK “ EMOTIONAL CONTAGION ” STUDY NOT FUN...
4,29hvyr,1404155395,2014-06-30 19:09:55,Password_is_123456,t2_6crtu,Microsoft May Prioritize The Desktop In Windows 9,http://techcrunch.com/2014/06/30/microsoft-may...,techcrunch.com,23,53,/r/technews/comments/29hvyr/microsoft_may_prio...,https://www.reddit.com/r/technews/comments/29h...,Microsoft May Prioritize The Desktop In Windows 9,Microsoft May Prioritize The Desktop In Windows,"[Microsoft, May, Prioritize, The, Desktop, In,...","[microsoft, may, priorit, the, desktop, in, wi...",microsoft may priorit the desktop in window,Microsoft May Desktop Windows


In [45]:
# add to list of stop words

# REFINE - add technology to stopwords
# REFINE - try without the top 10 tech companies as stop words

stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['new', 'news', 'tech', '&amp;', 'amp', 'million', 'billion',
                  'technology', 'world', 'number', 'technical', 
                 'google', 'apple', 'samsung', 'facebook', 'twitter', 'instagram', 
                 'amazon', 'microsoft', 'tesla', 'spacex', 
                 'xiaomi', 'huawei', 'kaspersky'] )


In [48]:
## topic modeling with title_nouns

vectorizer = CountVectorizer(stop_words = stopwords, ngram_range=(1,2))
title_words = vectorizer.fit_transform(posts.title_nouns)
title_words.shape

nmf_model = NMF(15)  # tried 5, 10, 12, 15, 17, 20, 25, 27, 30
post_topic = nmf_model.fit_transform(title_words)

words = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

[['android', 'apps', 'update', 'phones', 'android app', 'devices'],
 ['data', 'breach', 'data breach', 'users', 'center', 'data center'],
 ['iphone', 'iphone iphone', 'xs', 'ios', 'se', 'iphone xs'],
 ['windows', 'pc', 'download', 'mac', 'windows mac', 'pc windows'],
 ['app', 'store', 'mobile', 'app store', 'app development', 'mobile app'],
 ['security',
  'network',
  'cyber',
  'security services',
  'monitoring',
  'managed'],
 ['galaxy', 'note', 'galaxy note', 'update', 'fold', 'galaxy fold'],
 ['china', 'us', 'companies', 'trump', 'trade', 'company'],
 ['phone', 'mobile', 'cell', 'cell phone', 'android phone', 'camera'],
 ['ai', 'startup', 'musk', 'company', 'elon', 'elon musk'],
 ['internet', 'fcc', 'things', 'service', 'internet things', 'neutrality'],
 ['support', 'hp', 'printer', 'windows support', 'help', 'service'],
 ['pro', 'camera', 'india', 'price', 'redmi', 'smartphone'],
 ['services',
  'development',
  'software',
  'market',
  'development services',
  'app developmen

In [ ]:
# 12 looks pretty good, ai, startup, elon musk - in AI, ML topic
# still too many other topics under with these

In [ ]:
# 15 topics looks really good with the additional stop words
# the 2 apps topics can be combined
# the 3 cell phone topics can be combined


In [31]:
posts['post_topic_nmf'] = post_topic.argmax(axis=1)


In [32]:
#posts[posts['post_topic_nmf'] == 9].head()

In [33]:
post_info = posts.reset_index()

In [35]:
# add in the topic components

components = pd.DataFrame(post_topic.round(5),
             index = posts.title_nouns,
             columns = ["C_0", "C_1","C_2", "C_3", "C_4", "C_5", "C_6", "C_7", "C_8", "C_9", 
                       "C_10", "C_11", "C_12", "C_13", "C_14"])

c_info = components.reset_index()

c_info.head()

,title_nouns,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,C_11,C_12,C_13,C_14
0,New leaks iPhone s display,0.00000,0.00000,0.09853,0.00000,0.00000,0.00000,0.00401,0.00000,0.00141,0.00000,0.00000,0.00000,0.01020,0.00000,0.00000
1,Millions DNS users Microsoft IP domains,0.00416,0.01353,0.00191,0.01132,0.00852,0.00424,0.00000,0.00479,0.00216,0.00000,0.01647,0.00000,0.00000,0.00083,0.00000
2,Paypal Freezes ProtonMail Campaign Funds,0.00001,0.00058,0.00000,0.00011,0.00030,0.00060,0.00000,0.00120,0.00009,0.00053,0.00141,0.00028,0.00013,0.00019,0.00052
3,FACEBOOK “ EMOTIONAL CONTAGION ” STUDY NOT FUN...,0.00000,0.00164,0.00113,0.00186,0.00346,0.00348,0.00128,0.00256,0.00015,0.00849,0.00847,0.00380,0.00000,0.00000,0.00166
4,Microsoft May Desktop Windows,0.00000,0.00000,0.00000,0.16850,0.00000,0.00000,0.00000,0.00000,0.00020,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [36]:
c_info.shape

(48082, 16)

In [37]:
post_info.shape

(48082, 20)

In [38]:
# add the topic components to the full df

all_df = pd.concat([post_info, c_info], axis=1)
all_df.shape
        

(48082, 36)

In [39]:
# the CountVectorizer + NMF yielded pretty separate and easily labeled topics
# will examine if any benefit with clustering topic components

with open('cv_nmf_topics_posts_FINAL.pickle', 'wb') as to_write:
    pickle.dump(all_df, to_write)

In [40]:
all_df.columns

Index(['index', 'id', 'created_utc', 'date_time', 'author', 'author_fullname',
       'title', 'url', 'domain', 'num_comments', 'score', 'permalink',
       'full_link', 'refine_title', 'preproc_title', 'unstemmed', 'stem_title',
       'processed_title', 'title_nouns', 'post_topic_nmf', 'title_nouns',
       'C_0', 'C_1', 'C_2', 'C_3', 'C_4', 'C_5', 'C_6', 'C_7', 'C_8', 'C_9',
       'C_10', 'C_11', 'C_12', 'C_13', 'C_14'],
      dtype='object')

In [41]:
select_columns = all_df[['date_time', 'title', 'post_topic_nmf', 'C_0', 'C_1', 'C_2', 'C_3', 'C_4', 'C_5', 'C_6', 'C_7', 'C_8', 'C_9',
       'C_10', 'C_11', 'C_12', 'C_13', 'C_14']]

In [42]:
select_columns.head()

,date_time,title,post_topic_nmf,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,C_11,C_12,C_13,C_14
0,2014-07-01 03:19:52,New leaks show off the iPhone 6's curved edged...,2,0.00000,0.00000,0.09853,0.00000,0.00000,0.00000,0.00401,0.00000,0.00141,0.00000,0.00000,0.00000,0.01020,0.00000,0.00000
1,2014-07-01 03:19:51,Millions of dynamic DNS users suffer after Mic...,10,0.00416,0.01353,0.00191,0.01132,0.00852,0.00424,0.00000,0.00479,0.00216,0.00000,0.01647,0.00000,0.00000,0.00083,0.00000
2,2014-07-01 02:28:42,Paypal Freezes ProtonMail Campaign Funds,10,0.00001,0.00058,0.00000,0.00011,0.00030,0.00060,0.00000,0.00120,0.00009,0.00053,0.00141,0.00028,0.00013,0.00019,0.00052
3,2014-06-30 19:30:44,UNETHICAL FACEBOOK “EMOTIONAL CONTAGION” STUDY...,9,0.00000,0.00164,0.00113,0.00186,0.00346,0.00348,0.00128,0.00256,0.00015,0.00849,0.00847,0.00380,0.00000,0.00000,0.00166
4,2014-06-30 19:09:55,Microsoft May Prioritize The Desktop In Windows 9,3,0.00000,0.00000,0.00000,0.16850,0.00000,0.00000,0.00000,0.00000,0.00020,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [43]:
select_columns.to_excel('Review_FINAL_topics.xlsx', index=False)

In [49]:
all_df.to_excel('Review_ALL_FINAL_topics.xlsx', index=False)

/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:950: UserWarning: Ignoring URL 'https://www.reddit.com/r/technews/comments/4vq2jq/fcc_forces_tplink_to_support_open_source_firmware/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:950: UserWarning: Ignoring URL 'https://www.reddit.com/r/technews/comments/4vpj1h/bbc_iplayer_streaming_loophole_to_close_on_1/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:950: UserWarning: Ignoring URL 'https://www.reddit.com/r/technews/comments/4vnzqz/with_another_high_profile_dnc_email_hack_in_the/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel'

/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:950: UserWarning: Ignoring URL 'https://www.reddit.com/r/technews/comments/bn2fwo/here_comes_the_worlds_first_aigenerated_whisky/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:950: UserWarning: Ignoring URL 'https://www.reddit.com/r/technews/comments/bn200u/how_to_think_about_inclusion_in_tech_with_chanda/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:950: UserWarning: Ignoring URL 'https://www.reddit.com/r/technews/comments/bn1pjy/competition_lowers_internet_costs_but_blockchain/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Exce

/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:950: UserWarning: Ignoring URL 'https://www.reddit.com/r/technews/comments/b1u7o6/apple_watch_app_could_save_your_life_by_detecting/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:950: UserWarning: Ignoring URL 'https://www.reddit.com/r/technews/comments/b1u57p/this_ycbacked_startup_preps_chinese_students_for/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:950: UserWarning: Ignoring URL 'https://www.reddit.com/r/technews/comments/b1thwx/startups_weekly_ubers_headlinegrabbing_week_and/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Ex

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:950: UserWarning: Ignoring URL 'https://www.cnet.com/news/spacex-falcon-heavy-cant-stick-the-triple-landing-after-most-difficult-launch-ever/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
/Users/amybutler/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:950: UserWarning: Ignoring URL 'https://www.techradar.com/news/htc-launches-two-new-phones-surprising-everyone' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
/User